<a href="https://colab.research.google.com/github/rladbstj56/2024_PythonDataAnalysis/blob/main/241114LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import numpy as np
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/pythondataanalysis/Sleep_health_and_lifestyle_dataset.csv')
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Person ID                374 non-null    int64  
 1   Gender                   374 non-null    object 
 2   Age                      374 non-null    int64  
 3   Occupation               374 non-null    object 
 4   Sleep Duration           374 non-null    float64
 5   Quality of Sleep         374 non-null    int64  
 6   Physical Activity Level  374 non-null    int64  
 7   Stress Level             374 non-null    int64  
 8   BMI Category             374 non-null    object 
 9   Blood Pressure           374 non-null    object 
 10  Heart Rate               374 non-null    int64  
 11  Daily Steps              374 non-null    int64  
 12  Sleep Disorder           155 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 38.1+ KB


In [57]:
df1 = pd.concat([df, df['Blood Pressure'].str.split('/',expand = True)],axis = 1).drop('Blood Pressure', axis = 1)
df1 = df1.rename(columns = {0:'BloodPressure_Upper_Value',1:'BloodPressure_Lower_Value'})
df1.head()
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Person ID                  374 non-null    int64  
 1   Gender                     374 non-null    object 
 2   Age                        374 non-null    int64  
 3   Occupation                 374 non-null    object 
 4   Sleep Duration             374 non-null    float64
 5   Quality of Sleep           374 non-null    int64  
 6   Physical Activity Level    374 non-null    int64  
 7   Stress Level               374 non-null    int64  
 8   BMI Category               374 non-null    object 
 9   Heart Rate                 374 non-null    int64  
 10  Daily Steps                374 non-null    int64  
 11  Sleep Disorder             155 non-null    object 
 12  BloodPressure_Upper_Value  374 non-null    object 
 13  BloodPressure_Lower_Value  374 non-null    object 

In [58]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
for feature in ['Gender','Occupation','BMI Category','Sleep Disorder']:
  df1[feature] = label_encoder.fit_transform(df1[feature])
  print(label_encoder.classes_)
df1.head()
df1.info()

['Female' 'Male']
['Accountant' 'Doctor' 'Engineer' 'Lawyer' 'Manager' 'Nurse'
 'Sales Representative' 'Salesperson' 'Scientist' 'Software Engineer'
 'Teacher']
['Normal' 'Normal Weight' 'Obese' 'Overweight']
['Insomnia' 'Sleep Apnea' nan]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Person ID                  374 non-null    int64  
 1   Gender                     374 non-null    int64  
 2   Age                        374 non-null    int64  
 3   Occupation                 374 non-null    int64  
 4   Sleep Duration             374 non-null    float64
 5   Quality of Sleep           374 non-null    int64  
 6   Physical Activity Level    374 non-null    int64  
 7   Stress Level               374 non-null    int64  
 8   BMI Category               374 non-null    int64  
 9   Heart Rate                 374 non

In [59]:
# 이상치 제거 전
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

x = df1.drop('Sleep Disorder',axis = 1, inplace = False)
y = df1['Sleep Disorder']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 42)
model = LogisticRegression(random_state=1)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
acc = accuracy_score(y_test,y_pred)
print("Accuracy : ",acc)

Accuracy :  0.76


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [60]:
from sklearn.metrics import confusion_matrix
cf = confusion_matrix(y_test,y_pred)
print(cf)

[[10  1  5]
 [ 2 11  3]
 [ 6  1 36]]


In [61]:
num_col = ['Age','Sleep Duration','Quality of Sleep',
'Physical Activity Level', 'Stress Level','Heart Rate',
'Daily Steps','BloodPressure_Upper_Value','BloodPressure_Lower_Value']

# Convert 'BloodPressure_Upper_Value' and 'BloodPressure_Lower_Value' to numeric
df1['BloodPressure_Upper_Value'] = pd.to_numeric(df1['BloodPressure_Upper_Value'])
df1['BloodPressure_Lower_Value'] = pd.to_numeric(df1['BloodPressure_Lower_Value'])

Q1 = df1[num_col].quantile(0.25) #1사분위
Q3 = df1[num_col].quantile(0.75) #3사분위
IQR = Q3 - Q1
#이상치가 있는 열이 하나라도 있으면 해당 행은 필터링 제거
#df1= df1[~((df1[num_col]<(Q1-1.5*IQR)) | (df1[num_col]>(Q3+1.5*IQR))).any(axis=1)]
df1 = df1[((df1[num_col]>=(Q1-1.5*IQR)) & (df1[num_col]<=(Q3+1.5*IQR))).all(axis=1)]
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 359 entries, 0 to 373
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Person ID                  359 non-null    int64  
 1   Gender                     359 non-null    int64  
 2   Age                        359 non-null    int64  
 3   Occupation                 359 non-null    int64  
 4   Sleep Duration             359 non-null    float64
 5   Quality of Sleep           359 non-null    int64  
 6   Physical Activity Level    359 non-null    int64  
 7   Stress Level               359 non-null    int64  
 8   BMI Category               359 non-null    int64  
 9   Heart Rate                 359 non-null    int64  
 10  Daily Steps                359 non-null    int64  
 11  Sleep Disorder             359 non-null    int64  
 12  BloodPressure_Upper_Value  359 non-null    int64  
 13  BloodPressure_Lower_Value  359 non-null    int64  
dtyp

In [62]:
df2 = df1[['Physical Activity Level','Daily Steps','BloodPressure_Lower_Value','Age','BloodPressure_Upper_Value','Quality of Sleep','Heart Rate']]
df1.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Heart Rate,Daily Steps,Sleep Disorder,BloodPressure_Upper_Value,BloodPressure_Lower_Value
0,1,1,27,9,6.1,6,42,6,3,77,4200,2,126,83
1,2,1,28,1,6.2,6,60,8,0,75,10000,2,125,80
2,3,1,28,1,6.2,6,60,8,0,75,10000,2,125,80
7,8,1,29,1,7.8,7,75,6,0,70,8000,2,120,80
8,9,1,29,1,7.8,7,75,6,0,70,8000,2,120,80


In [63]:
x = df2
y = df1['Sleep Disorder']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 42)
model = LogisticRegression(random_state=1)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
acc = accuracy_score(y_test,y_pred)
print("Accuracy : ",acc)

Accuracy :  0.7638888888888888


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [64]:
cf = confusion_matrix(y_test,y_pred)
print(cf)

[[ 4  0  7]
 [ 0 11  2]
 [ 8  0 40]]


In [ ]:
corr = df1.corr()
print(corr['Sleep Disorder'].sort_values(ascending=False))
df2 = df1[['Sleep Disorder','Physical Activity Level','Daily Steps']]